In [250]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(1, os.path.dirname(os.getcwd()))

from scipy import fft
import numpy as np
from utils.network_graph import DirectedGraph, graph_dict_3x3, graph_dict_3x3_2, graph_dict_4_nodes


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [251]:
graph = DirectedGraph(graph_dict_3x3)
# graph = DirectedGraph(graph_dict_4_nodes)


In the next cell you can see the process of restoring od-flows from o-flows and getting link flows using both O and OD flows and assignment matrices. Fortunately, the results are equivalent.

In [252]:
o_flows, o_nodes = graph.generate_o_flows()
# print(fft.dct(o_flows, type=1, norm="forward"))
A_od = graph.generate_od_assignment_matrix()
P_o = graph.generate_o_assignment_matrix()

od_flows, od_pairs = graph.get_od_from_o_flows(o_flows, P_o)

print("P_o.shape:", P_o.shape, end="; ")
print("A_od.shape:", A_od.shape, end="\n\n")

print("o flows: \n", o_flows)
print("od flows: \n", np.round(od_flows))

print("\nlink flows from o: \n", np.round(P_o @ o_flows))
print("link flows from od: \n", np.round(A_od @ od_flows))


P_o.shape: (12, 8); A_od.shape: (12, 27)

o flows: 
 [846 319 683 790 276 996 863 328]
od flows: 
 [109. 161.   1. 138. 168.   7. 118. 144.  58.  42.  76.  89.  53. 472.
 211. 200. 111. 169. 161. 149. 115. 125.  35. 996. 276. 587. 328.]

link flows from o: 
 [ 562.  284.  354.  417.  818.  703.  370.  509.  507. 1382. 1057. 1122.]
link flows from od: 
 [ 562.  284.  354.  417.  818.  703.  370.  509.  507. 1382. 1057. 1122.]


In [253]:
# example solution of a task posed in
# https://scaron.info/blog/quadratic-programming-in-python.html

from utils.opt_functions import quadprog_solve_qp

M = np.array([[1., 2., 0.], [-8., 3., 2.], [0., 1., 1.]])
P = np.dot(M.T, M)
q = -np.dot(M.T, np.array([3., 2., 3.]))
G = np.array([[1., 2., 1.], [2., 0., 1.], [-1., 2., -1.]])
h = np.array([3., 2., -2.])

quadprog_solve_qp(P, q, G, h)

# array([ 0.12997347, -0.06498674,  1.74005305])

array([ 0.12997347, -0.06498674,  1.74005305])

In [254]:
# optimize o-flows with a known assignment matrix
M = P_o
P = np.dot(M.T, M)
q = -np.dot(M.T, P_o @ o_flows) # P_o @ o_flows is a vector of link flows
G = -np.ones((len(o_flows), len(o_flows))) # C1: o-flows should be positive
h = np.zeros(len(o_flows)) 

print("o-flows opt solution:", quadprog_solve_qp(P, q, G, h))
print("o-flows ground truth:", o_flows)

o-flows opt solution: [846. 319. 683. 790. 276. 996. 863. 328.]
o-flows ground truth: [846 319 683 790 276 996 863 328]


In [255]:
# test that some of constraints are encoded properly.
# They are!
# C3_mat @ P_o[:].reshape((r*c,))

array([2.09988198, 1.5914946 , 1.09752502, 0.24361652, 0.75481948,
       0.57174098, 0.46046794, 0.47747887])